This assignment involves the different querry problems related to 3 database released by the state of 
Chicago: the socio-economic indices, public schools & crime rates. The data will be used to create sql tables and solve the problems that follow.

In [7]:
#un-comment if the libraries are not already installed
#!pip install pandas, ipython-sql
import pandas as pd
import csv, sqlite3

In [5]:
#create the database or establish the connection if already exists
con = sqlite3.connect('Database1.db')
#create the cursor object
cur = con.cursor()

#load the sql magic command provided by the ipython-sql library
%load_ext sql
#connect sql to the database
%sql sqlite:///Database1.db

In [8]:
#use pandas to read the csv files
df1 = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01')
df2 = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01')
df3 = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01')

#create sql tables out of the above data
df1.to_sql('CENSUS', con, if_exists='replace', index=False, method='multi')
df2.to_sql('SCHOOLS', con, if_exists='replace', index=False, chunksize=100)
df3.to_sql('CRIME', con, if_exists='replace', index=False, method='multi')

533

In [55]:
#look at the column names of the crime table
%sql PRAGMA TABLE_INFO(CENSUS)

 * sqlite:///Database1.db
Done.


cid,name,type,notnull,dflt_value,pk
0,COMMUNITY_AREA_NUMBER,REAL,0,None,0
1,COMMUNITY_AREA_NAME,TEXT,0,None,0
2,PERCENT_OF_HOUSING_CROWDED,REAL,0,None,0
3,PERCENT_HOUSEHOLDS_BELOW_POVERTY,REAL,0,None,0
4,PERCENT_AGED_16__UNEMPLOYED,REAL,0,None,0
5,PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA,REAL,0,None,0
6,PERCENT_AGED_UNDER_18_OR_OVER_64,REAL,0,None,0
7,PER_CAPITA_INCOME,INTEGER,0,None,0
8,HARDSHIP_INDEX,REAL,0,None,0


In [59]:
%sql SELECT * FROM CRIME LIMIT 2;

 * sqlite:///Database1.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
3512276,HK587712,2004-08-28,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,0,0,911,9,14.0,58.0,6,1155838.0,1873050.0,2004,41.8074405,-87.70395585,"(41.8074405, -87.703955849)"
3406613,HK456306,2004-06-26,009XX N CENTRAL PARK AVE,820,THEFT,$500 AND UNDER,OTHER,0,0,1112,11,27.0,23.0,6,1152206.0,1906127.0,2004,41.89827996,-87.71640551,"(41.898279962, -87.716405505)"


Problem 1
Find the total number of crimes recorded in the CRIME table.

In [12]:
%sql SELECT COUNT(*) FROM CRIME;

 * sqlite:///Database1.db
Done.


COUNT(*)
533


Problem 2
List community area names and numbers with per capita income less than 11000.

In [18]:
#this info is in the census table
%sql SELECT COMMUNITY_AREA_NAME, COMMUNITY_AREA_NUMBER FROM CENSUS WHERE PER_CAPITA_INCOME < 11000

 * sqlite:///Database1.db
Done.


COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER
West Garfield Park,26.0
South Lawndale,30.0
Fuller Park,37.0
Riverdale,54.0


Problem 3
List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)

In [44]:
#we can search for 'minor' in the description column using the LIKE operator
%sql SELECT CASE_NUMBER FROM CRIME WHERE DESCRIPTION LIKE '%MINOR%'

 * sqlite:///Database1.db
Done.


CASE_NUMBER
HL266884
HK238408


Problem 4
List all kidnapping crimes involving a child?

In [46]:
%sql SELECT CASE_NUMBER FROM CRIME WHERE PRIMARY_TYPE = 'KIDNAPPING' AND DESCRIPTION LIKE '%CHILD%'

 * sqlite:///Database1.db
Done.


CASE_NUMBER
HN144152


Problem 5
List the kind of crimes that were recorded at schools. (No repetitions)

In [50]:
%sql SELECT PRIMARY_TYPE FROM CRIME WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%' GROUP BY PRIMARY_TYPE

 * sqlite:///Database1.db
Done.


PRIMARY_TYPE
ASSAULT
BATTERY
CRIMINAL DAMAGE
CRIMINAL TRESPASS
NARCOTICS
PUBLIC PEACE VIOLATION


Problem 6
List the type of schools along with the average safety score for each type.

In [54]:
%sql SELECT [Elementary, Middle, or High School] AS SCHOOL_TYPE, AVG(SAFETY_SCORE) FROM SCHOOLS GROUP BY SCHOOL_TYPE

 * sqlite:///Database1.db
Done.


SCHOOL_TYPE,AVG(SAFETY_SCORE)
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


Problem 7
List 5 community areas with highest % of households below poverty line

In [58]:
%sql SELECT COMMUNITY_AREA_NAME FROM CENSUS ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5;

 * sqlite:///Database1.db
Done.


COMMUNITY_AREA_NAME
Riverdale
Fuller Park
Englewood
North Lawndale
East Garfield Park


Problem 8
Which community area is most crime prone? Display the coumminty area number only

In [67]:
%sql SELECT COMMUNITY_AREA_NUMBER, COUNT(COMMUNITY_AREA_NUMBER) AS CRIME_FREQ FROM CRIME GROUP BY COMMUNITY_AREA_NUMBER ORDER BY CRIME_FREQ DESC LIMIT 1;

 * sqlite:///Database1.db
Done.


COMMUNITY_AREA_NUMBER,CRIME_FREQ
25.0,43


Problem 9
Use a sub-query to find the name of the community area with highest hardship index

In [68]:
%sql SELECT COMMUNITY_AREA_NAME FROM CENSUS WHERE HARDSHIP_INDEX = (SELECT MAX(HARDSHIP_INDEX) FROM CENSUS)

 * sqlite:///Database1.db
Done.


COMMUNITY_AREA_NAME
Riverdale


Use a sub-query to determine the Community Area Name with most number of crimes?

In [72]:
%sql SELECT COMMUNITY_AREA_NAME FROM CENSUS WHERE COMMUNITY_AREA_NUMBER = (SELECT COMMUNITY_AREA_NUMBER FROM CRIME GROUP BY COMMUNITY_AREA_NUMBER ORDER BY COUNT(COMMUNITY_AREA_NUMBER) DESC LIMIT 1)

 * sqlite:///Database1.db
Done.


COMMUNITY_AREA_NAME
Austin
